# Langgraph

### 0- Packages needed

| **Package**    | **Purpose**                                                                   |
| -------------- | ----------------------------------------------------------------------------- |
| `langgraph`    | Core library for building dynamic graphs of function calls with state passing |
| `langchain`    | Provides unified interfaces for LLMs, tools, chains, and pipelines            |
| `transformers` | From Hugging Face – used to load and run local LLMs                           |
| `accelerate`   | Helps efficiently load large models across CPU/GPU                            |
| `einops`       | Required by many Hugging Face models for tensor reshaping                     |


**Steps:**
Build a simple LangGraph-based question-answering agent that:

1- Understands a user’s question

2- Decides if a math tool is needed

3- Calls a calculator tool (if required)

4- Generates a final response

It will run entirely locally using a HuggingFace LLM (e.g., Mistral-7B-Instruct) with no external APIs.


### 1- Loading a local llm

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
import torch

# Load tokenizer and model (replace with your local path if needed)
model_id = "tiiuae/falcon-rw-1b" # simple and fast model for testing

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Create a text-generation pipeline and wrap it with LangChain
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
local_llm = HuggingFacePipeline(pipeline=pipe)


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

c:\code\projects\gr\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rigia\.cache\huggingface\hub\models--tiiuae--falcon-rw-1b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

Device set to use cpu
C:\Users\rigia\AppData\Local\Temp\ipykernel_29612\2112411347.py:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  local_llm = HuggingFacePipeline(pipeline=pipe)


### 2- Define the agent state

We define a shared state to pass information between LangGraph nodes.

In [3]:
from typing import TypedDict, List

class AgentState(TypedDict):
    input: str
    steps: List[str]
    tool_input: str
    tool_output: str
    final_answer: str


| **Field**      | **Type**    | **Description**                                                           | **When It's Set / Updated**                                              | **Used By**                           |
| -------------- | ----------- | ------------------------------------------------------------------------- | ------------------------------------------------------------------------ | ------------------------------------- |
| `input`        | `str`       | The original user question or prompt                                      | Initially set when invoking the graph                                    | All nodes (decision, synthesis, etc.) |
| `steps`        | `List[str]` | (Optional) Logs intermediate steps, decisions, or tool calls              | Can be appended to at each step for trace/debugging (optional use)       | Optional: for tracing/debug/debugging |
| `tool_input`   | `str`       | The expression or command extracted from the input to be passed to a tool | Set in `extract_tool_input` node (if tool is used)                       | `call_tool`                           |
| `tool_output`  | `str`       | The result returned from the tool (e.g., evaluated math expression)       | Set in `call_tool` node                                                  | `synthesize`                          |
| `final_answer` | `str`       | The final response generated by the LLM agent                             | Set in `synthesize` node (based on `input` and optionally `tool_output`) | Final output returned to the user     |


**Defining a calculator**

In [4]:
def calculator(expression: str) -> str:
    try:
        return str(eval(expression))
    except:
        return "Could not evaluate expression."


### 3- Create Nodes (functions in the graph)

Each node is a function and takes/returns agentState


In [5]:
# 3-1 handle_input
def handle_input(state: AgentState) -> AgentState:
    print("📥 User input:", state["input"])
    return state


# 3-2 decide_tool
def decide_tool(state: AgentState) -> str:
    prompt = f"""
    You are an assistant. Does this question require a math calculation?

    Question: {state['input']}

    Respond with 'use_tool' or 'skip_tool'.
    """
    response = local_llm(prompt)
    decision = "use_tool" if "use_tool" in response.lower() else "skip_tool"
    print("🧭 Decision:", decision)
    return decision


# 3-3 extract_tool_input
def extract_tool_input(state: AgentState) -> AgentState:
    prompt = f"""
    Extract the math expression from this question: "{state['input']}".
    Just return the expression, nothing else.
    """
    expression = local_llm(prompt).strip()
    state["tool_input"] = expression
    return state


# 3-4 call_tool
def call_tool(state: AgentState) -> AgentState:
    expr = state.get("tool_input", "")
    state["tool_output"] = calculator(expr)
    return state

# 3-5 finalize_answer

def synthesize(state: AgentState) -> AgentState:
    if state.get("tool_output"):
        prompt = f"""
        The user asked: "{state['input']}".
        You used a tool and got: {state['tool_output']}.
        Provide the final answer.
        """
    else:
        prompt = f"Answer the question: {state['input']}"
    
    final = local_llm(prompt).strip()
    state["final_answer"] = final
    return state


### 4- Building LangGraph

In [11]:
from langgraph.graph import StateGraph, END, START

graph = StateGraph(AgentState)

graph.add_node("input_handler", handle_input)
graph.add_node("extract_tool_input", extract_tool_input)
graph.add_node("call_tool", call_tool)
graph.add_node("synthesize", synthesize)

# Add the entrypoint
graph.add_edge(START, "input_handler")

# Add conditional routing logic
graph.add_conditional_edges(
    "input_handler",
    decide_tool,
    {
        "use_tool": "extract_tool_input",
        "skip_tool": "synthesize"
    }
)

# Normal transitions
graph.add_edge("extract_tool_input", "call_tool")
graph.add_edge("call_tool", "synthesize")
graph.add_edge("synthesize", END)

# Compile the graph
agent_executor = graph.compile()
print(agent_executor.get_graph().draw_ascii())


                   +-----------+          
                   | __start__ |          
                   +-----------+          
                          *               
                          *               
                          *               
                 +---------------+        
                 | input_handler |        
                 +---------------+        
                 ...            ...       
               ..                  ..     
             ..                      ..   
+--------------------+                 .. 
| extract_tool_input |                  . 
+--------------------+                  . 
           *                            . 
           *                            . 
           *                            . 
    +-----------+                      .. 
    | call_tool |                    ..   
    +-----------+                  ..     
                 ***            ...       
                    **        ..          
           

### 5- Run the Agent

In [12]:
initial_state = {
    "input": "What is 12 * (5 + 3)?",
    "steps": [],
    "tool_input": "",
    "tool_output": "",
    "final_answer": ""
}

final_state = agent_executor.invoke(initial_state)
print("\n✅ Final Answer:", final_state["final_answer"])


C:\Users\rigia\AppData\Local\Temp\ipykernel_29612\2316854134.py:16: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = local_llm(prompt)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📥 User input: What is 12 * (5 + 3)?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🧭 Decision: use_tool


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



✅ Final Answer: The user asked: "What is 12 * (5 + 3)?".
        You used a tool and got: Could not evaluate expression..
        Provide the final answer.


## Using LangGraph with multiple knowledgebases

Using LangGraph to orchestrate different RAG systems/databases for different domains is a very good idea, especially when:

| **Reason**                                      | **Explanation**                                                                                                                          |
| ----------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------- |
| **Modular routing**                             | LangGraph allows you to dynamically choose which retriever/knowledge base to use based on the user input or detected domain.             |
| **Separation of concerns**                      | Each RAG system can have its own chunking logic, embeddings, and vector store tuned for its domain (e.g., legal, medical, product info). |
| **Scalable and maintainable**                   | Easier to maintain or swap out domain-specific RAG pipelines without affecting others.                                                   |
| **Tool selection logic is native to LangGraph** | You can use conditional routing (`add_conditional_edges`) to switch between domain-specific RAG branches.                                |
| **Reusability and extension**                   | You can add fallback paths (e.g., general RAG or LLM-only) if domain-specific RAG yields low-confidence results.                         |


User Input  
&nbsp;&nbsp;&nbsp;&nbsp;↓  
detect_domain  
&nbsp;&nbsp;&nbsp;&nbsp;├── medical → medical_retrieve → medical_synthesize  
&nbsp;&nbsp;&nbsp;&nbsp;├── legal → legal_retrieve → legal_synthesize  
&nbsp;&nbsp;&nbsp;&nbsp;├── ecommerce → product_retrieve → product_synthesize  
&nbsp;&nbsp;&nbsp;&nbsp;└── unknown → fallback_synthesize

